In [63]:
import pandas as pd
import numpy as np

In [83]:
df = pd.read_csv('/content/EX4.csv')
df


,Age,Sex,Bp,Cholesterol,Drug
0,Young,F,High,Normal,A
1,Young,F,High,High,A
2,Middle-age,F,High,Normal,B
3,Senior,F,Normal,Normal,B
4,Senior,M,Low,Normal,B
5,Senior,M,Low,High,A
6,Middle-age,M,Low,High,B
7,Young,F,Normal,Normal,A
8,Young,M,Low,Normal,B
9,Senior,M,Normal,Normal,B


In [65]:
import math
def find_entrophy(a,b,n):
  ent = 0
  for i in range(len(a)):
    c = 0
    for j in range(len(b)):
      if(b.iloc[j] == a[i]):
        c=c+1
    if(c!=0):
      p = c/n
      ent = ent - (p*(math.log2(p)))
  return ent

In [66]:
def entropy(data, target):
    entropy = 0
    for value in set(data[target]):
        #send = df[df[column_headers[i]] == unique[j]]
        p = len(data[data[target] == value]) / len(data)
        entropy += -p * math.log2(p)
    return entropy

In [67]:
def find_target_entrophy(data,attribute,target):
  a = entropy(data,target)
  return a

In [68]:
def information_gain(data, attribute, target):
    info =[]
    head = find_target_entrophy(data,attribute,target)
    for value in set(data[attribute]):
        subset = data[data[attribute] == value]
        head -= len(subset) / len(data) * entropy(subset, target)
        '''for j in range(len(unique)):
           send = df[df[column_headers[i]] == unique[j]]
           a = find_entrophy(unique_chars,send['Drug'],len(send))
           entropy = entropy + ((len(send) / len(df)) * a)
           info.append(head - (entropy))'''
    return head

In [69]:
def Build_tree(data, target, attributes):
    if len(set(data[target])) == 1:
        return data[target].iloc[0]

    if len(attributes) == 0:
        return data[target].value_counts().idxmax()

    best_attribute = max(attributes, key=lambda attribute: information_gain(data, attribute, target))
    tree = {best_attribute: {}}
    for value in set(data[best_attribute]):
        subset = data[data[best_attribute] == value]
        if len(subset) == 0:
            tree[best_attribute][value] = data[target].value_counts().idxmax()
        else:
            new_attributes = [a for a in attributes if a != best_attribute]
            subtree = Build_tree(subset, target, new_attributes)
            tree[best_attribute][value] = subtree
    return tree

In [70]:
attributes = ['Age ', 'Sex', 'Bp', 'Cholesterol']
target = 'Drug'
tree = Build_tree(df, target, attributes)
print(tree)

{'Age ': {'Young': {'Sex': {'M': 'B', 'F': 'A'}}, 'Senior': {'Cholesterol': {'High': 'A', 'Normal': 'B'}}, 'Middle-age': 'B'}}


In [84]:
df1 = pd.read_csv('/content/train.csv')
df1

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [85]:
df1.drop('Cabin', inplace=True, axis=1)
df1.drop(['Name', 'PassengerId', 'Ticket'], axis=1, inplace=True)
df1['family_size'] = df1.SibSp + df1.Parch+1
df1.drop(['SibSp', 'Parch'], axis=1, inplace=True)
df1['Sex'] = df1['Sex'].map({'male': 0, 'female': 1})
df1['Embarked'] = df1['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df1['Age'].fillna(df1['Age'].median(), inplace=True)
df1['Embarked'].fillna(df1['Embarked'].mode()[0], inplace=True)
df1

,Survived,Pclass,Sex,Age,Fare,Embarked,family_size
0,0,3,0,22.0,7.2500,0.0,2
1,1,1,1,38.0,71.2833,1.0,2
2,1,3,1,26.0,7.9250,0.0,1
3,1,1,1,35.0,53.1000,0.0,2
4,0,3,0,35.0,8.0500,0.0,1
...,...,...,...,...,...,...,...
886,0,2,0,27.0,13.0000,0.0,1
887,1,1,1,19.0,30.0000,0.0,1
888,0,3,1,28.0,23.4500,0.0,4
889,1,1,0,26.0,30.0000,1.0,1


In [86]:
df1.drop(['Age','Fare'], axis=1, inplace=True)


In [87]:
df1

,Survived,Pclass,Sex,Embarked,family_size
0,0,3,0,0.0,2
1,1,1,1,1.0,2
2,1,3,1,0.0,1
3,1,1,1,0.0,2
4,0,3,0,0.0,1
...,...,...,...,...,...
886,0,2,0,0.0,1
887,1,1,1,0.0,1
888,0,3,1,0.0,4
889,1,1,0,1.0,1


In [88]:
attributes = ['Pclass','Sex','Embarked','family_size']
target = 'Survived'
tree = Build_tree(df1, target, attributes)
print(tree)

{'Sex': {0: {'family_size': {1: {'Pclass': {1: {'Embarked': {0.0: 0, 1.0: 0}}, 2: {'Embarked': {0.0: 0, 1.0: 0, 2.0: 0}}, 3: {'Embarked': {0.0: 0, 1.0: 0, 2.0: 0}}}}, 2: {'Pclass': {1: {'Embarked': {0.0: 0, 1.0: 0}}, 2: {'Embarked': {0.0: 0, 1.0: 0}}, 3: {'Embarked': {0.0: 0, 1.0: 0, 2.0: 0}}}}, 3: {'Pclass': {1: {'Embarked': {0.0: 1, 1.0: 0, 2.0: 0}}, 2: {'Embarked': {0.0: 0, 1.0: 0}}, 3: {'Embarked': {0.0: 0, 1.0: 0, 2.0: 0}}}}, 4: {'Pclass': {1: 1, 2: {'Embarked': {0.0: 0, 1.0: 0}}, 3: {'Embarked': {0.0: 0}}}}, 5: 0, 6: 0, 7: {'Pclass': {3: {'Embarked': {0.0: 0}}}}, 8: 0, 11: 0}}, 1: {'Pclass': {1: {'family_size': {1: {'Embarked': {0.0: 1, 1.0: 1}}, 2: 1, 3: 1, 4: {'Embarked': {0.0: 0}}, 5: 1, 6: 1}}, 2: {'family_size': {1: {'Embarked': {0.0: 1, 1.0: 1, 2.0: 1}}, 2: {'Embarked': {0.0: 1, 1.0: 1}}, 3: {'Embarked': {0.0: 1}}, 4: 1, 5: 1, 6: 1}}, 3: {'family_size': {1: {'Embarked': {0.0: 0, 1.0: 1, 2.0: 1}}, 2: {'Embarked': {0.0: 0, 1.0: 0, 2.0: 1}}, 3: {'Embarked': {0.0: 1, 1.0: 1, 2.